In [1]:
from transformer_nb2 import *
from dataset import make_data_generator
import json
from tqdm import tqdm_notebook as tqdm
#from torchsummary import summary

In [2]:
folder = 'data/food/'
data_name = folder+'data.json'
# validation_name = folder+'valid_seq.json'
# testdata_name = folder+'testdata_seq.json'
vocab_name = folder+'vocab.json'

In [3]:
num_epochs = 10
save_rate = 1 #how many epochs per modelsave
#continue_from = "trained/Model1" # if none, put None
continue_from = None
epsilon = 1e-8
validation_size = 10000
device = torch.device('cuda')
!mkdir -p trained

In [4]:
vocab = json.load(open(vocab_name, 'r'))
VOC_SIZE = len(vocab)
INPUT_MAX = 100
SUMM_MAX = 100
UNK = "[UNK]"
BOS = "[CLS]"
EOS = "[SEP]"
PAD = "[PAD]"

In [5]:
batch_size = 64

training_set, training_generator = make_data_generator(\
data_name, INPUT_MAX, SUMM_MAX, vocab[PAD], batch_size, pretrain=True, cutoff=None, shuffle=True, num_workers=4)

# validation_set, validation_generator = make_data_generator(\
# validation_name, INPUT_MAX, OUTPUT_MAX, vocab[PAD], batch_size, cutoff=validation_size, shuffle=False, num_workers=4)

def data_gen_train():
    for src, tgt in training_generator:
        src = src.to(device)
        tgt = tgt.to(device)
        b = Batch(src, tgt, vocab[PAD])
        yield b

loading json
load json done.



(tensor([ 1012,  1996,   101,  1997,  3899,  9120,  2488,  2066,  1012,  6799,
         2031,  1012,  2022,  3688,  2062,  1998,  1037,  2009,  3737,  1996,
         2084,  3504,  4149,  2488,  4031,  1998,  4031,  1997,  2035,   102,
        13995,  2833,  8995,  2100,  2084,  2000,  2031,  2015, 18604, 20717,
        27141,  3012,  2026,  2179,  6240,  2068,  2016,  2087, 14747,  1037,
         1045,  2204,  1998,  2003, 10346,  2195,  2023,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0]), tensor([  101,  1045,  2031,  4149,  2195,  1997,  1996,  8995,  3012, 27141,
         3899,  2833,  3688,  1998,  2031,  2179,  2068,  2035,  2000,  2022,
         1997,  2204,  3737,  1012,  1996,  4031,  3504,  20

In [6]:
import math
total_train = int(math.ceil(training_set.size / batch_size))
# total_valid = int(math.ceil(validation_set.size / batch_size))
# print(total_train, total_valid)

In [7]:
def init_param(model):
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

# translator = PointerGenerator(
#             hidden_dim=256, 
#             emb_dim=256, 
#             input_len=INPUT_MAX, 
#             output_len=SUMM_MAX, 
#             voc_size=VOC_SIZE, 
#             eps=1e-8
#         ).to(device)
translator = make_model(VOC_SIZE, VOC_SIZE, N=2, d_model=64, d_ff=1024, h=8, dropout=0.1, emb_share=True).to(device)

criterion = LabelSmoothing(size=VOC_SIZE, padding_idx=vocab[PAD], smoothing=0.1).to(device)
model_opt = torch.optim.Adam(translator.parameters(), lr=1e-4, betas=(0.9, 0.998), eps=1e-8)
loss_compute = SimpleLossCompute(translator.generator, criterion, model_opt)

/home/marvin/anaconda3/envs/py37cuda10/lib/python3.7/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [8]:
# import torch.nn.functional as F
# def loss_compute(x, y, norm): 
#     x = F.log_softmax(x)
    
#     print(x.shape)
    
#     loss = criterion(x.contiguous().view(-1, x.size(-1)), 
#                           y.contiguous().view(-1)) / norm
#     loss.backward()
#     if model_opt is not None:
#         model_opt.step()
#         model_opt.zero_grad()
#     return loss.item() * norm

In [ ]:
# model = make_big_bird(vocab, N=4, d_model=256, d_ff=512, h=8, dropout=0.1, emb_share=True, bert_share=True)
#model.load("Nest/NewbornBird")

In [ ]:
vocab_inv = {a:b for b, a in vocab.items()}
def convert_ids_to_tokens(ids):
    return [vocab_inv[i] for i in ids]

In [ ]:
start = 1 if continue_from == None else (int(continue_from.split("Model")[-1])+1)
history = []


for epoch in range(start, num_epochs+1):
    print("Epoch", epoch)
    
    # training
    stats = Stats()
    translator.train()
    trange = tqdm(enumerate(data_gen_train()), total=total_train)
    
    for i, batch in trange:
        out = translator.forward(batch.src, batch.trg, batch.src_mask, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        stats.update(loss, batch.ntokens, log=1)
        
        if( i % 150 == 0):
            probs = translator.generator(out) 
            print("\n")
            next_words = torch.argmax(probs, dim=-1, keepdim=True)            
            print(convert_ids_to_tokens([i.item() for i in next_words[0]]))
        
        trange.set_postfix(
            **{'loss': '{:.3f}'.format(loss)}
        )
        stats.update(loss, 1, log=0)
        
    t_h = stats.history
    history.append(t_h)
    
    print("[info] epoch train loss:", np.mean(t_h))
    
    try:
        !mkdir -p pretrained
        torch.save({'model':translator.state_dict(), 'training_history':t_h}, 
                   "pretrained/Translator"+str(epoch))
    except:
        continue

Epoch 1


Step: 1 Loss: 8.965453 Tokens / Sec: 10575.903413

['briefing', 'theories', 'theories', 'theories', 'theories', 'theories', 'theories', 'theories', 'theories', 'theories', 'theories', '##osing', '152', '##osing', '##osing', '##osing', 'yamaha', 'theories', 'yamaha', 'malicious', '##osing', 'theories', 'theories', 'theories', 'theories', 'brighter', 'ak', 'ruin', 'ruin', 'ruin', 'ruin', 'ruin', 'yamaha', 'ruin', 'billed', 'ruin', 'ruin', 'rightful', 'theories', 'theories', 'yamaha', '##osing', 'billed', 'ruin', 'yamaha', 'collected', 'ruin', 'ruin', 'billed', 'theories', 'malicious', 'quota', 'yamaha', 'rows', 'yamaha', 'yamaha', '##osing', '##osing', 'theories', 'concrete', 'contrasting', 'theories', 'theories', 'theories', 'theories', 'rows', 'malicious', 'theories', 'theories', 'yamaha', 'concrete', 'theories', 'coaster', 'yamaha', 'yamaha', 'theories', 'declines', '152', 'theories', 'theories', 'theories', 'theories', 'contrasting', 'theories', 'yamaha', 'ak', 'billed', 'theories', 

Step: 3901 Loss: 4.763590 Tokens / Sec: 49862.602260

['i', '.', 'i', 'is', '.', 'a', '.', 'i', "'", 'the', 'little', '-', '-', '.', '.', '.', 'the', 'favorite', '.', '.', '.', '.', '.', 'i', 'i', "'", "'", '.', 'i', "'", 'to', 'the', 'the', "'", '.', 'i', 'i', 'i', "'", 'the', 'of', 'a', '.', 's', 'a', 'the', 'best', '.', 'i', "'", 'a', '.', 'the', 'the', 'flavor', 'of', 'the', '.', '.', '.', 'i', 'best', '.', 'i', 'the', 'best', '.', '.', '.', '.', 'i', "'", 'a', "'", '.', 'the', 'best', '.', 'i', '.', 'the', 'the', 'i', 'best', '.', 'a', 'to', 'be', 'the', 'i', 'best', '.', 'a', '.', 'i', 'best', '.', 'a', 'the']
Step: 4201 Loss: 4.618985 Tokens / Sec: 54140.754969

['i', 'a', 'i', 'i', 'is', '.', 'the', 'is', '.', 'the', 'i', 'i', "'", 'the', 'is', 'the', 'dog', '.', '.', '.', 'i', "'", 's', 'a', 'the', '.', 'i', "'", 'i', "'", 'the', 'is', 'the', 'favorite', '.', '.', 'a', '.', '.', 'i', '.', 's', 'a', '.', 'the', 'little', '.', 'the', '.', '.', 'i', '.', '.', 'i', 'i', '.', 'i', 

Step: 7801 Loss: 4.336039 Tokens / Sec: 52521.095408

['i', 'love', 'this', 'i', 'this', 'i', 'this', 'product', '.', 'i', 'i', 'i', 'is', 's', 'a', 'of', "'", 'this', 'the', 'of', 'favorite', ',', ',', 'the', 'i', '.', '.', '.', 'i', '.', 'i', 'like', 'the', 'the', 'of', 'best', '.', '.', 'the', 'i', ',', 'i', 'product', 'of', 'i', "'", 'it', "'", 'i', 'a', 'i', 'of', 'the', 'it', '.', '.', 'i', '##s', 'it', 'best', '.', 'i', 'have', 'it', '##i', '.', 'i', 'it', '.', 'the', 'favorite', '-', '.', 'i', '.', '.', 'have', 't', 'not', 'i', 'i', "'", 'it', '##ing', "'", 'i', '.', 't', '.', 'little', '.', 'it', 'i', 'not', "'", '.', "'", 'it', 'is']
Step: 8101 Loss: 4.287855 Tokens / Sec: 51635.632917

['this', 'best', 'is', 'is', 'the', 'best', '##uri', '##uri', '##ories', ',', 'the', 'is', '##at', '##af', '##d', '.', 'a', '.', 'the', 'are', 'like', 'not', 'is', 'the', 'little', '##mmy', '##p', '##o', ',', '-', '.', '.', 'to', 'the', 'you', 'are', 'a', 'to', 'the', 'to', 'little', 'more', '

Step: 11401 Loss: 4.072031 Tokens / Sec: 48557.272459

['i', "'", 've', 'not', 'little', 'bit', 'of', 'these', '-', ',', '##ing', 'and', 'i', 'a', 'good', 'to', 'find', 'these', 'little', 'favorite', '.', 'i', "'", 'a', 'i', 'are', 'be', 'a', 'much', ',', 'a', "'", 've', 'not', 'like', 'little', 'and', 'and', '.', 'and', 'i', 'are', 'not', 'i', '##mmy', '!', 'i', 'flavor', '.', 'them', '.', '[SEP]', '.', '!', '!', '!', '!', '.', '!', '!', '!', '!', '!', '[SEP]', '!', '!', '!', '!', '.', '!', '!', '!', '!', '!', '!', '[SEP]', '!', '[SEP]', '.', '[SEP]', '[SEP]', '!', '[SEP]', '.', '!', '.', '!', '!', '.', '!', '[SEP]', '!', '!', '!', '[SEP]', '[SEP]', '!', '!', '.']
Step: 11701 Loss: 4.261196 Tokens / Sec: 54831.608810

['i', 'dog', 'loves', 'this', 'product', 'i', 'product', 'be', 'the', 'the', '.', 'i', 'had', 'a', 'to', 'of', 'a', 'to', 'my', '.', 'and', 'the', '.', '.', 'and', ')', "'", ')', '.', 'food', ')', '.', 'first', 'food', ')', 'have', 'to', 'a', 'the', 'dog', '.', 'i', 'the

...

Step: 14401 Loss: 3.122669 Tokens / Sec: 46088.120246

['i', 'was', 've', 'been', 'using', 'a', '##ug', '##ree', '##k', '##p', 'my', '2', 'weeks', '.', 'i', 'i', "'", 'the', '!', 'i', 'works', 'great', 'than', 'i', "'", 'a', '##ug', '##ree', '##k', 'for', 'i', "'", 've', 'a', 'day', 'out', 'who', 'was', 'to', 'days', '3', 'times', 'a', 'day', '.', 'my', '.', 'i', 'love', 'worried', 'about', 'i', "'", "'", 't', 'keep', 'able', 'to', 'keep', 'my', 'to', 'my', 'kids', '.', 'to', 'i', 'son', 'time', 'days', '.', 'to', 'first', '.', 'day', '.', 'the', 'than', 'first', '.', '.', 'i', "'", '.', '.', 'i', 'a', 'few', 'days', 'i', 'my', 'my', 'day', '##ug', '##ree', '##k', 'i', "'", 'able', 'to']
Step: 14701 Loss: 3.065906 Tokens / Sec: 48812.617588

['they', 'product', 'for', 'i', 'they', 'of', 'chips', 'chips', 'are', 'don', 'don', 'don', "'", 't', 'think', 'they', 'are', 'broken', 'for', 'shipping', '!', 'shipping', 'or', 'shipping', 'shipping', '.', '[SEP]', '!', '!', '[SEP]', '!', '[SEP]',

Step: 17765 Loss: 2.943547 Tokens / Sec: 18722.549264
[info] epoch train loss: 7161.673108023109
Epoch 6


Step: 1 Loss: 3.137419 Tokens / Sec: 17543.107766

['this', 'bought', 'this', 'for', 'for', 'my', 'my', 'great', 'of', 'i', 'product', 'are', 'my', 'has', 'i', 'has', 'me', 'to', 'my', 'blood', 'and', 'i', 'a', 'a', 'problems', 'with', 'my', 'blood', '.', 'a', '.', '##ring', '.', 'this', '.', '##s', '.', 'i', 'product', 'is', 'a', 'to', 'my', 'vet', 'skin', '##er', '##gies', 'and', 'is', 'will', 'to', 'to', 'use', 'it', 'product', 'to', 'me', 'i', 'will', 'continue', 'to', 'find', 'this', 'product', '.', 'times', '.', 'day', '.', '[SEP]', 'this', '[SEP]', '[SEP]', '[SEP]', '[SEP]', '[SEP]', '[SEP]', '.', '[SEP]', '[SEP]', '[SEP]', '[SEP]', 'service', '[SEP]', '[SEP]', '[SEP]', '[SEP]', '[SEP]', '[SEP]', '.', '[SEP]', '[SEP]', '[SEP]', '[SEP]', '[SEP]', '[SEP]', '[SEP]', '[SEP]', '[SEP]']
Step: 301 Loss: 3.187212 Tokens / Sec: 54755.807056

['we', 'soup', 'very', 'first', 'time', 'trying', 'it', 'other', 'soup', 'and', 'it', 'have', 'very', 'happy', 'with', 'it', '.', 'and', 'we', 'was'

Step: 3601 Loss: 2.944290 Tokens / Sec: 51690.184432

['i', 'ordered', 'this', 'chocolate', 'chocolate', 'chocolate', 'chocolate', 'chocolate', 'chocolate', 'chocolate', 'chocolate', '##eme', 'chocolate', 'is', 'chocolate', 'i', 'very', 'disappointed', '.', 'i', 'ordered', 'the', 'order', '.', 'i', 'chocolate', 'arrived', 'very', 'melted', 'and', 'the', 'of', '.', 'disappointed', '.', 'i', 'was', 'no', 'added', 'cream', 'to', 'the', 'package', 'to', 'the', 'the', 'get', 'you', 'the', 'package', 'in', 'the', '.', 'the', 'ordered', 'the', 'and', 'the', 'items', 'and', 'amazon', '.', 'they', 'the', 'website', '.', '.', 'the', 'shape', '.', 'but', 'the', 'was', 'very', '.', 'i', 'would', 'm', 'very', 'to', 'package', 'was', 'be', 'to', 'to', 'you', 'arrived', '.', 'not', '.', '.', '.', 'i', 'wish', 'your', 'you', 'get', 'these', 'amazon']
Step: 3901 Loss: 3.027809 Tokens / Sec: 47143.304129

['i', 'bought', 'a', 'got', 'this', 'i', 'coffee', 'coffee', 'is', 'sell', 'as', 'good', 'as', 'a',

Step: 7201 Loss: 3.054234 Tokens / Sec: 50554.118269

['my', 'daughter', 'liked', 'a', 'the', 'best', 'family', 'of', 'my', 'daughter', 'daughter', 'ate', 'the', 'of', 'these', 'best', 'and', 'she', 'ate', 'the', 'butter', 'and', 'than', 'the', 'and', 'and', 'they', 'are', 'cal', '-', '##ories', '##e', '.', '##es', '.', 'the', 'the', 'right', 'amount', 'of', 'the', 'taste', 'great', '.', '[SEP]', 'my', '.', '.', 'for', '.', '.', 'for', '[SEP]', '.', '.', '.', '.', 'and', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '##es', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
Step: 7501 Loss: 3.179012 Tokens / Sec: 50807.802743

['i', 'the', 'the', 'tea', 'is', '##s', 'and', 'use', 'used', 'it', 'like', 'it', 'like', 'the', 'tea', '.', 'best', '.', 'it', 'small', 'and', 'take', 'it', 'it', 'up', 'with', 'it', 'in', 'a', 'up', 'you', 'are', 'i', 'to', 'it', 'i', 'it', 'a', 'than', 'the',

In [ ]:
history = np.asarray(history).flatten()